<a href="https://colab.research.google.com/github/brianpyai/chatbot/blob/main/Llama_3_1_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install gradio==4.22.0 transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:

#@title Built llama-cpp-python with BLAS or CUDA automatically, wait for few minutes .
import os,torch


if torch.cuda.is_available ():
    cmd='CMAKE_ARGS="%s" pip install llama-cpp-python==0.2.62' % "-DLLAMA_CUBLAS=on"
else:
    cmd='CMAKE_ARGS="%s" pip install llama-cpp-python==0.2.62' % "-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"

os.system (cmd)

0

In [ ]:

from IPython import get_ipython
ipython = get_ipython()




from llama_cpp import Llama,LlamaRAMCache,LlamaDiskCache,LlamaTokenizer,LlamaState
from llama_cpp.llama_speculative import LlamaPromptLookupDecoding
from llama_cpp.llama_chat_format import Llava15ChatHandler
import os,torch,json,shlex
from pathlib import Path
from PIL import Image
from huggingface_hub import hf_hub_download

try:
  import google.colab
  IN_COLAB = True
except :IN_COLAB=False
Gbase="./generate/"
cache_dir="./hf/"
from IPython.display import display, Markdown
import torch
from psutil import cpu_count
import platform
if torch.cuda.is_available():
    n_gpu_layers=-1
    n_threads=cpu_count()
else :
    n_gpu_layers=0
    n_threads=cpu_count()

if platform.machine()=='aarch64' and not IN_COLAB:n_threads=4


description ="""<div style="font-family: Arial, sans-serif; padding: 20px;">

This showcases the use of llama-cpp-python to load different versions of 4-bit and 8-bit models, as well as a simple chat interface created by Gradio. It can run on small computers or even mobile devices and produce satisfactory results. You are free to modify this code. I will continue to update these tools, first unlocking more possibilities to allow this tool to handle most common content, and then combining all of these possibilities with automation elements. Ultimately, it will evolve into a powerful tool that solves real-world problems effectively.

這裡展示了使用llama-cpp-python來加載各種4位和8位模型的不同版本，以及由Gradio創建的簡單聊天界面。它可以在小型電腦甚至手機上運行，並能產生令人滿意的結果。您可以自由修改這段代碼。我將繼續更新這些工具，首先解鎖更多可能性，使此工具能夠處理大多數常見內容，然後將所有這些可能性與自動化元素結合在一起。最終，它將發展成一個有效解決現實問題的工具。
  <h2>🔗My social media links❤️</h2>

Follow <a href="https://www.facebook.com/braiml"  ytarget="_blank">🐍Brian's Page </a> if you want I share more tools .<br>
Follow<a href="https://www.facebook.com/charactersAI" target="_blank">❤️Characters AI</a>
 if you want more  videos .
<br>
  <a href="https://www.facebook.com/brian.pyai" target="_blank"> 📘facebook.com/brian.pyai</a>
 <br>
  <a href="https://www.facebook.com/braiml" target="_blank">🐍Brian's Page </a>
<br>

  <a href="https://www.facebook.com/lovelyai999" target="_blank">🥰AI Hot Shorts </a>



</div>

"""
import base64
import io
def image_to_base64_data_uri(file_path):
    i=Image.open(file_path)
    i.resize((256,256))
    img_byte_arr = io.BytesIO()
    i.save(img_byte_arr, format='PNG')
    base64_data = base64.b64encode(img_byte_arr.getvalue()).decode('utf-8')
    return f"data:image/png;base64,{base64_data}"

modelsPath="./"



modelsPaths= ["lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"  , "lmstudio-community/Meta-Llama-3.1-70B-Instruct-GGUF/Meta-Llama-3.1-70B-Instruct-IQ2_M.gguf" , "mradermacher/llava-v1.6-mistral-7b-GGUF/llava-v1.6-mistral-7b.Q4_K_M.gguf","RDson/llava-llama-3-8b-v1_1-GGUF/llava-llama-3-8b-v1_1-Q8_0.gguf","IHaveNoClueAndIMustPost/Llama-3-11.5B-Instruct-v2_GGUF/Replete-AI_Llama-3-11.5B-Instruct-v2-Q6_K.gguf","PrunaAI/Llama-3-16B-GGUF-smashed/Llama-3-16B.Q8_0.gguf","PrunaAI/Llama-3-16B-GGUF-smashed/Llama-3-16B.Q4_K_M.gguf", "SanctumAI/Phi-3-mini-4k-instruct-GGUF/phi-3-mini-4k-instruct.Q4_K_M.gguf","mradermacher/Llama3-8B-DPO-uncensored-GGUF/Llama3-8B-DPO-uncensored.Q8_0.gguf","mradermacher/Llama3-Inst-8B-DPO-Ultrafeedback-GGUF/Llama3-Inst-8B-DPO-Ultrafeedback.Q8_0.gguf","DavidAU/Llama3-8B-OpenHermes-DPO-Q8_0-GGUF/llama3-8b-openhermes-dpo.Q8_0.gguf","LoneStriker/OrpoLlama-3-8B-GGUF/OrpoLlama-3-8B-Q8_0.gguf","MaziyarPanahi/WizardLM-2-7B-GGUF/WizardLM-2-7B.Q4_K_M.gguf","SanctumAI/Phi-3-mini-4k-instruct-GGUF/phi-3-mini-4k-instruct.Q8_0.gguf","MaziyarPanahi/WizardLM-2-7B-GGUF/WizardLM-2-7B.Q8_0.gguf","FaradayDotDev/llama-3-8b-Instruct-GGUF/llama-3-8b-Instruct.Q4_K_M.gguf","seyf1elislam/llama-3-neural-chat-v1-8b-GGUF/llama-3-neural-chat-v1-8b.Q4_K_M.gguf","seyf1elislam/llama-3-neural-chat-v1-8b-GGUF/llama-3-neural-chat-v1-8b.Q8_0.gguf","Quant-Cartel/Llama-3-8B-Instruct-DADA-iMat-GGUF/Llama-3-8B-Instruct-DADA-iMat-IQ4_XS.gguf","Quant-Cartel/Llama-3-8B-Instruct-DADA-iMat-GGUF/Llama-3-8B-Instruct-DADA-iMat-Q8_0.gguf","3thn/dolphin-2.9-llama3-8b-GGUF/dolphin-2.9-llama3-8b.Q4_K_M.gguf","3thn/dolphin-2.9-llama3-8b-GGUF/dolphin-2.9-llama3-8b.Q8_0.gguf","PawanKrd/Meta-Llama-3-70B-Instruct-GGUF/llama-3-70b-instruct.Q3_K_M.gguf","twodgirl/zephyr-beta-wizardLM-2-merge-7B-Q6_K-GGUF/zephyr-beta-wizardlm-2-merge-7b.Q6_K.gguf","MaziyarPanahi/WizardLM-2-8x22B-GGUF/WizardLM-2-8x22B.IQ1_M.gguf","TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/dolphin-2.7-mixtral-8x7b.Q2_K.gguf","TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/dolphin-2.7-mixtral-8x7b.Q4_K_M.gguf","mradermacher/Starling-LM-alpha-8x7B-MoE-GGUF/Starling-LM-alpha-8x7B-MoE.Q4_K_M.gguf","MaziyarPanahi/Qwen1.5-8x7b-v0.1-GGUF/Qwen1.5-8x7b-v0.1.Q4_K_M.gguf","TheBloke/firefly-mixtral-8x7b-GGUF/firefly-mixtral-8x7b.Q4_K_M.gguf","TheBloke/openbuddy-mixtral-8x7b-v15.1-GGUF/openbuddy-mixtral-8x7b-v15.1.Q4_K_M.gguf","Quant-Cartel/Cerebrum-1.0-8x7b-iMat-GGUF/Cerebrum-1.0-8x7b-iMat-Q4_K_M.gguf","MaziyarPanahi/Experiment26-7B-GGUF/Experiment26-7B.Q4_K_M.gguf","dagbs/dolphin-2.8-experiment26-7b-preview-GGUF/dolphin-2.8-experiment26-7b.Q4_K_M.gguf","ggml-org/gemma-1.1-2b-it-Q4_K_M-GGUF/gemma-1.1-2b-it.Q4_K_M.gguf","chenhunghan/TAIDE-LX-7B-Chat-GGUF/taide-lx-7b-chat.q4_k_m.gguf","chenhunghan/TAIDE-LX-7B-Chat-GGUF/taide-lx-7b-chat.q8_0.gguf","ggml-org/gemma-1.1-7b-it-Q4_K_M-GGUF/gemma-1.1-7b-it.Q4_K_M.gguf","ggml-org/gemma-1.1-2b-it-Q8_0-GGUF/gemma-1.1-2b-it.Q8_0.gguf","ggml-org/gemma-1.1-7b-it-Q8_0-GGUF/gemma-1.1-7b-it.Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/gemma-2b-GGUF/gemma-2b-Q4_K_M.gguf","LoneStriker/gemma-2b-it-GGUF/gemma-2b-it-Q4_K_M.gguf","LoneStriker/gemma-7b-GGUF/gemma-7b-Q4_K_M.gguf","LoneStriker/gemma-7b-it-GGUF/gemma-7b-it-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/OrcaGemma-2B-GGUF/OrcaGemma-2B-Q4_K_M.gguf","rombodawg/EveryoneLLM-7b-Gemma-Base-GGUF/EveryoneLLM-7b-Gemma-Base-q6_k.gguf","LoneStriker/openbuddy-gemma-7b-v19.1-4k-GGUF/openbuddy-gemma-7b-v19.1-4k-Q4_K_M.gguf","LoneStriker/Gemmalpaca-7B-GGUF/Gemmalpaca-7B-Q4_K_M.gguf", "LoneStriker/zephyr-7b-gemma-v0.1-GGUF/zephyr-7b-gemma-v0.1-Q4_K_M.gguf" ,"Lewdiculous/firefly-gemma-7b-GGUF-IQ-Imatrix/firefly-gemma-7b-Q4_K_S-imatrix.gguf"]



model_id="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf" #@param    ["lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf" , "lmstudio-community/Meta-Llama-3.1-70B-Instruct-GGUF/Meta-Llama-3.1-70B-Instruct-IQ2_M.gguf" , "mradermacher/llava-v1.6-mistral-7b-GGUF/llava-v1.6-mistral-7b.Q4_K_M.gguf","RDson/llava-llama-3-8b-v1_1-GGUF/llava-llama-3-8b-v1_1-Q8_0.gguf","IHaveNoClueAndIMustPost/Llama-3-11.5B-Instruct-v2_GGUF/Replete-AI_Llama-3-11.5B-Instruct-v2-Q6_K.gguf","PrunaAI/Llama-3-16B-GGUF-smashed/Llama-3-16B.Q8_0.gguf","PrunaAI/Llama-3-16B-GGUF-smashed/Llama-3-16B.Q4_K_M.gguf", "SanctumAI/Phi-3-mini-4k-instruct-GGUF/phi-3-mini-4k-instruct.Q4_K_M.gguf","mradermacher/Llama3-8B-DPO-uncensored-GGUF/Llama3-8B-DPO-uncensored.Q8_0.gguf","mradermacher/Llama3-Inst-8B-DPO-Ultrafeedback-GGUF/Llama3-Inst-8B-DPO-Ultrafeedback.Q8_0.gguf","DavidAU/Llama3-8B-OpenHermes-DPO-Q8_0-GGUF/llama3-8b-openhermes-dpo.Q8_0.gguf","LoneStriker/OrpoLlama-3-8B-GGUF/OrpoLlama-3-8B-Q8_0.gguf","MaziyarPanahi/WizardLM-2-7B-GGUF/WizardLM-2-7B.Q4_K_M.gguf","SanctumAI/Phi-3-mini-4k-instruct-GGUF/phi-3-mini-4k-instruct.Q8_0.gguf","MaziyarPanahi/WizardLM-2-7B-GGUF/WizardLM-2-7B.Q8_0.gguf","FaradayDotDev/llama-3-8b-Instruct-GGUF/llama-3-8b-Instruct.Q4_K_M.gguf","seyf1elislam/llama-3-neural-chat-v1-8b-GGUF/llama-3-neural-chat-v1-8b.Q4_K_M.gguf","seyf1elislam/llama-3-neural-chat-v1-8b-GGUF/llama-3-neural-chat-v1-8b.Q8_0.gguf","Quant-Cartel/Llama-3-8B-Instruct-DADA-iMat-GGUF/Llama-3-8B-Instruct-DADA-iMat-IQ4_XS.gguf","Quant-Cartel/Llama-3-8B-Instruct-DADA-iMat-GGUF/Llama-3-8B-Instruct-DADA-iMat-Q8_0.gguf","3thn/dolphin-2.9-llama3-8b-GGUF/dolphin-2.9-llama3-8b.Q4_K_M.gguf","3thn/dolphin-2.9-llama3-8b-GGUF/dolphin-2.9-llama3-8b.Q8_0.gguf","PawanKrd/Meta-Llama-3-70B-Instruct-GGUF/llama-3-70b-instruct.Q3_K_M.gguf","twodgirl/zephyr-beta-wizardLM-2-merge-7B-Q6_K-GGUF/zephyr-beta-wizardlm-2-merge-7b.Q6_K.gguf","MaziyarPanahi/WizardLM-2-8x22B-GGUF/WizardLM-2-8x22B.IQ1_M.gguf","TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/dolphin-2.7-mixtral-8x7b.Q2_K.gguf","TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/dolphin-2.7-mixtral-8x7b.Q4_K_M.gguf","mradermacher/Starling-LM-alpha-8x7B-MoE-GGUF/Starling-LM-alpha-8x7B-MoE.Q4_K_M.gguf","MaziyarPanahi/Qwen1.5-8x7b-v0.1-GGUF/Qwen1.5-8x7b-v0.1.Q4_K_M.gguf","TheBloke/firefly-mixtral-8x7b-GGUF/firefly-mixtral-8x7b.Q4_K_M.gguf","TheBloke/openbuddy-mixtral-8x7b-v15.1-GGUF/openbuddy-mixtral-8x7b-v15.1.Q4_K_M.gguf","Quant-Cartel/Cerebrum-1.0-8x7b-iMat-GGUF/Cerebrum-1.0-8x7b-iMat-Q4_K_M.gguf","MaziyarPanahi/Experiment26-7B-GGUF/Experiment26-7B.Q4_K_M.gguf","dagbs/dolphin-2.8-experiment26-7b-preview-GGUF/dolphin-2.8-experiment26-7b.Q4_K_M.gguf","ggml-org/gemma-1.1-2b-it-Q4_K_M-GGUF/gemma-1.1-2b-it.Q4_K_M.gguf","chenhunghan/TAIDE-LX-7B-Chat-GGUF/taide-lx-7b-chat.q4_k_m.gguf","chenhunghan/TAIDE-LX-7B-Chat-GGUF/taide-lx-7b-chat.q8_0.gguf","ggml-org/gemma-1.1-7b-it-Q4_K_M-GGUF/gemma-1.1-7b-it.Q4_K_M.gguf","ggml-org/gemma-1.1-2b-it-Q8_0-GGUF/gemma-1.1-2b-it.Q8_0.gguf","ggml-org/gemma-1.1-7b-it-Q8_0-GGUF/gemma-1.1-7b-it.Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/gemma-2b-GGUF/gemma-2b-Q4_K_M.gguf","LoneStriker/gemma-2b-it-GGUF/gemma-2b-it-Q4_K_M.gguf","LoneStriker/gemma-7b-GGUF/gemma-7b-Q4_K_M.gguf","LoneStriker/gemma-7b-it-GGUF/gemma-7b-it-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/OrcaGemma-2B-GGUF/OrcaGemma-2B-Q4_K_M.gguf","rombodawg/EveryoneLLM-7b-Gemma-Base-GGUF/EveryoneLLM-7b-Gemma-Base-q6_k.gguf","LoneStriker/openbuddy-gemma-7b-v19.1-4k-GGUF/openbuddy-gemma-7b-v19.1-4k-Q4_K_M.gguf","LoneStriker/Gemmalpaca-7B-GGUF/Gemmalpaca-7B-Q4_K_M.gguf", "LoneStriker/zephyr-7b-gemma-v0.1-GGUF/zephyr-7b-gemma-v0.1-Q4_K_M.gguf" ,"Lewdiculous/firefly-gemma-7b-GGUF-IQ-Imatrix/firefly-gemma-7b-Q4_K_S-imatrix.gguf"]

def selectPath(paths=modelsPaths):
    for i, t in enumerate(paths):
        print(f"{i}) {t}")
    print("輸入數字選擇模型:")
    v = paths[int(input())]
    return v

#model_id = selectPath()

# 下載模型文件
def downHgFile(url, targetDir=modelsPath):
    fileName = Path(url).name
    repo = url[:-len(fileName)-1]
    print(fileName, repo)
    fileExists = os.path.exists(os.path.join(targetDir, fileName))

    print(repo, fileName, fileExists)
    if not fileExists:
        print("正在下載文件:")
        hf_hub_download(repo, filename=fileName, local_dir=targetDir, local_dir_use_symlinks=False)

max_tokens = 4096
n_ctx = 4096

downHgFile(model_id)

# 設置模型路徑並加載模型
modelPath = os.path.join(modelsPath, Path(model_id).name)
if ("8x" in model_id or "70b" in model_id) and torch.cuda.is_available():
    n_gpu_layers = 16
    if "70b" in model_id:
        n_gpu_layers = 24
    n_threads = 4

if "/llava" in model_id:
    if not os.path.exists("mistral_7b_mmproj-v1_5_Q4_1.gguf"):
        hf_hub_download("lovelyai999/temp", filename="mistral_7b_mmproj-v1_5_Q4_1.gguf", local_dir="./", local_dir_use_symlinks=False)
    chat_handler = Llava15ChatHandler(clip_model_path="mistral_7b_mmproj-v1_5_Q4_1.gguf", verbose=True)
    model = Llama(modelPath, n_gpu_layers=n_gpu_layers, n_threads=n_threads, max_tokens=4096, logits_all=True, n_ctx=n_ctx, chat_handler=chat_handler)
else:
    model = Llama(modelPath, n_gpu_layers=n_gpu_layers, n_threads=n_threads, max_tokens=4096, logits_all=True, n_ctx=n_ctx)

tokenizer = model.tokenize

# 定義預測函數
def predict(message, top_k=300, top_p=0.95, temp=0.85, repeat_penalty=1.1, max_tokens=max_tokens):
    global partial_message,model_id
    try :messageT=message["text"]
    except :messageT=message
    global model_id

    if "phi-3-" in model_id:
        prompt=f"<|user|>{messageT}<|end|><|assistant|>"
    else:
        prompt=f"""### System:
You are a professional private assistant.
### User:
{messageT}
###  Response:

"""
    prompt_=f"""
###Below is an instruction that describes my question or task.
Write a response that appropriately completes the request:
{messageT}
### Response :

"""

    print (message)
    #model_inputs = tokenizer(prompt.encode("utf-8"))
    #generate_kwargs = dict(top_k=top_k, top_p=top_p, temp=temp, repeat_penalty=repeat_penalty)
    stop= ["<|end|>" ,"<|end_of_text|>", "<|im_end|>"  ]
    if model_id in [  "lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf" ,   "lmstudio-community/Phi-3.1-mini-4k-instruct-GGUF/Phi-3.1-mini-4k-instruct-Q8_0.gguf" , "lmstudio-community/Meta-Llama-3.1-70B-Instruct-GGUF/Meta-Llama-3.1-70B-Instruct-IQ2_M.gguf" ,   "FaradayDotDev/llama-3-8b-Instruct-GGUF/llama-3-8b-Instruct.Q4_K_M.gguf","l3utterfly/llama-3-8b-Instruct-gguf/llama-3-8b-Instruct-Q4_K.gguf","l3utterfly/llama-3-8b-Instruct-gguf/llama-3-8b-Instruct-Q8_0.gguf","PawanKrd/Meta-Llama-3-70B-Instruct-GGUFFaradayDotDev/llama-3-8b-Instruct-GGUF/llama-3-70b-instruct.Q2_K.gguf","PawanKrd/Meta-Llama-3-70B-Instruct-GGUF/llama-3-70b-instruct.Q3_K_M.gguf", "LoneStriker/OrpoLlama-3-8B-GGUF/OrpoLlama-3-8B-Q8_0.gguf" ,"mradermacher/Llama3-8B-DPO-uncensored-GGUF/Llama3-8B-DPO-uncensored.Q8_0.gguf","mradermacher/Llama3-Inst-8B-DPO-Ultrafeedback-GGUF/Llama3-Inst-8B-DPO-Ultrafeedback.Q8_0.gguf","DavidAU/Llama3-8B-OpenHermes-DPO-Q8_0-GGUF/llama3-8b-openhermes-dpo.Q8_0.gguf","RDson/llava-llama-3-8b-v1_1-GGUF/llava-llama-3-8b-v1_1-Q8_0.gguf","IHaveNoClueAndIMustPost/Llama-3-11.5B-Instruct-v2_GGUF/IHaveNoClueAndIMustPost/Replete-AI_Llama-3-11.5B-Instruct-v2-Q6_K.gguf","PrunaAI/Llama-3-16B-GGUF-smashed/Llama-3-16B.Q8_0.gguf","PrunaAI/Llama-3-16B-GGUF-smashed/Llama-3-16B.Q4_K_M.gguf" ]:
        stop=["### Below",".\n\n","assistant\n\n","!\n\n","<|end|>" ,"<|end_of_text|>" ,"<|im_end|>","System:"   ]
    generate_kwargs=dict (suffix=None, max_tokens=max_tokens, temperature=temp, top_p=top_p, min_p=0.05, typical_p=1.0, logprobs=None, echo=False, stop=stop, frequency_penalty=0,presence_penalty=0.0, repeat_penalty=1.1, top_k=top_k, stream=True , seed=None, tfs_z=1.0, mirostat_mode=0, mirostat_tau=5.0, mirostat_eta=0.1, model=None, stopping_criteria=None, logits_processor=None, grammar=None, logit_bias=None)

    partial_message = ""
    if isinstance(message,dict) and message["files"] and message["files"][0]["path"] and message["files"][0]['mime_type'].startswith("image"):
        messages = [
        {"role": "system", "content": "You are an assistant who perfectly describes images."},
        {
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url":message["files"][0]["path"]}},
                {"type" : "text", "text": messageT}
            ]
        }]
        outputs =model.create_completion(json.dumps (messages),**generate_kwargs)
    else:
        outputs =model.create_completion(prompt ,**generate_kwargs)


    for chunk in outputs:
        #print (chunk )
        content = chunk["choices"][0]["text"]
        #print(content,repr(content ))
        if content:
            partial_message+=content
            yield partial_message

from IPython.display import display, Markdown, clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

def create_conversation_markdown(user_input, ai_response):
    return f"""
**You:** {user_input}

**AI:** {ai_response}

---
"""

print("歡迎使用AI聊天助手！輸入 'quit' 或 'exit' 結束對話。")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['quit', 'exit']:
        print("謝謝使用，再見！")
        break

    ai_response = ""
    # 創建初始顯示
    display_id = display(Markdown(create_conversation_markdown(user_input, ai_response)), display_id=True)

    for chunk in predict(user_input):
        ai_response += chunk
        # 更新顯示
        display_id.update(Markdown(create_conversation_markdown(user_input, ai_response)))

    # 在完成後清除輸出並顯示最終結果
    clear_output(wait=True)
    display(Markdown(create_conversation_markdown(user_input, ai_response)))


**You:** List all districts in Hong Kong .

**AI:** HereHere isHere is theHere is the listHere is the list ofHere is the list of districtsHere is the list of districts inHere is the list of districts in HongHere is the list of districts in Hong KongHere is the list of districts in Hong Kong :

Here is the list of districts in Hong Kong :

1Here is the list of districts in Hong Kong :

1.Here is the list of districts in Hong Kong :

1. CentralHere is the list of districts in Hong Kong :

1. Central andHere is the list of districts in Hong Kong :

1. Central and WesternHere is the list of districts in Hong Kong :

1. Central and Western DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
Here is the list of districts in Hong Kong :

1. Central and Western District
2Here is the list of districts in Hong Kong :

1. Central and Western District
2.Here is the list of districts in Hong Kong :

1. Central and Western District
2. WanHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan ChHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan ChaiHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. EasternHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. SouthernHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. KHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. KowHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. KowloonHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon CityHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. WongHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong TaiHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai SinHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. KwHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. KwunHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun TongHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. ShamHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham ShHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham ShuiHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui PoHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. YHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. YauHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau THere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau TsimHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim MongHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. TaiHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai PoHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. NorthHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. ShaHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha TinHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. TsHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. TsuenHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen WanHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. THere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. TuenHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen MunHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen Mun DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen Mun District
Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen Mun District
15Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen Mun District
15.Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen Mun District
15. YHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen Mun District
15. YuenHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen Mun District
15. Yuen LongHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen Mun District
15. Yuen Long DistrictHere is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen Mun District
15. Yuen Long District

Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen Mun District
15. Yuen Long District

###Here is the list of districts in Hong Kong :

1. Central and Western District
2. Wan Chai District
3. Eastern District
4. Southern District
5. Kowloon City District
6. Wong Tai Sin District
7. Kwun Tong District
8. Sham Shui Po District
9. Yau Tsim Mong District
10. Tai Po District
11. North District
12. Sha Tin District
13. Tsuen Wan District
14. Tuen Mun District
15. Yuen Long District

### 

---
